In [1]:
import numpy as np


In [14]:
# Elementary matrix : In mathematics, an elementary matrix is a matrix which differs from the identity matrix by one single elementary row operation
# 1. Row switching
# 2. Row multiplication
# 3, Row addition

# case1 switch two rows
def switch_rows(A, i, j):
    m,n = A.shape
    E = np.eye(m)
    E[i,i] = 0; E[j,j] = 0
    E[i,j] = 1; E[j,i] = 1
    return E @ A

x= np.arange(12).reshape(3,4)
print(x)
print(switch_rows(x, 0, 1))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 4.  5.  6.  7.]
 [ 0.  1.  2.  3.]
 [ 8.  9. 10. 11.]]


In [20]:
# permutation matrix to switch rows
def switch_rows_permutation(A, source_list, target_list):
    m,n = A.shape
    P = np.eye(m)
    for i in range(len(source_list)):
        P[source_list[i], source_list[i]] = 0
        P[target_list[i], target_list[i]] = 0
        P[source_list[i], target_list[i]] = 1
        P[target_list[i], source_list[i]] = 1
    return P @ A
x = np.arange(16).reshape(4,4)
print(x)
switch_rows_permutation(x, [0,1],[3,2])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


array([[12., 13., 14., 15.],
       [ 8.,  9., 10., 11.],
       [ 4.,  5.,  6.,  7.],
       [ 0.,  1.,  2.,  3.]])

In [4]:
# communication matrix for transpose matrix
# x[i,j] = x[i*n+j] -> 对于Python, matlab按行展开
x = np.arange(12).reshape(4,3)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        print(f"{x[i,j]} , {x.flatten()[i*x.shape[1]+j]}")

In [5]:
def transpose_by_commuinication_matrix(x):
    m, n = x.shape
    y = np.zeros((m*n,m*n))
    x_flatten = x.flatten()
    for i in range(m):
        for j in range(i,n):
            if i == j:
              # 如果是对角线元素，则不改变，因此在communication matrix中[i*n+j,i*n+j] = 1
              y[i*n+j,i*n+j] = 1
            else:
              # tranpose x[i,j] and x[j,i]
              # 如果非对角线元素，则交换x[i,j]与x[j,i]- > x[i,j]在flatten中为i*n+j行，x[j,i]在flatten中为j*n+i行，因此是i*n+j,j*n+i=1表示第i*n+j行的元素是原来第j*n+i行，j*n+i,i*n+1=1第j*n+i行的元素是原来第i*n+j行
              y[i*n+j,j*n+i] = 1
              y[j*n+i,i*n+j] = 1
    return y.dot(x_flatten).reshape(n,m),y

In [3]:
x = np.arange(9).reshape(3,3)
print(x)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [6]:
y, y_comm = transpose(x)

In [10]:
x.T == y

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [20]:
m,n = x.shape

In [21]:
m

3

In [ ]:
for i in range